<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2b4915abfba4630686ae916e3b4da4138312cf0c39a1e4f5172999eb75d6cc37
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 14:42:21
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 75.73 K (0.54%)
Current PnL: -19.66 L (-13.1%)
CY Booked + Current PnL: -8.22 L (-5.48%)
-------------------
Total profit:  1.88 L
Total loss:  -21.54 L
-------------------
Total Booked + Current PnL: 18.76 L (15.23%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.06%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.17 L (61.42%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.56%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.28,119165.0,18661.0,10451.0,-2.05,18.57,8.77,28.96,79.0,1.79,0.84,58.84,MH,ATH,METALS
14,BLUESTARCO,2080.00,12.85,59.0,X-MC,3.22,194850.0,30180.0,13152.0,2.97,18.33,6.75,26.31,89.0,2.29,1.38,27.77,X40N,NTT,AC
19,CIPLA,1662.74,-19.78,58.0,X-LC,3.68,213898.0,9398.0,13903.0,3.19,4.60,6.50,11.39,10.0,0.68,1.51,13.96,X40N,ATH,PHARMA
84,VOLTAS,1530.00,1.72,63.0,X-MC,1.10,213405.0,21663.0,16091.0,2.95,11.30,7.54,19.69,99.0,1.35,1.51,18.86,XY25,NTT,AC
77,TTKPRESTIG,770.00,93.17,39.0,M-SC,1.01,83185.0,-17592.0,17685.0,-0.32,-17.46,21.26,0.09,245.0,-0.99,0.59,9.29,OX40N,NTT,DURABLES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-6.72,40.0,H-SC,6.23,111271.0,-14026.0,72949.0,4.68,-11.19,65.56,47.03,141.0,-0.19,0.79,39.32,MH,ATH,POWER
63,SFL,1287.00,32.84,48.0,M-SC,7.20,175440.0,-86778.0,156598.0,3.30,-33.09,89.26,26.63,229.0,-0.55,1.24,9.08,XY24,NTT,MISC
19,CIPLA,1662.74,-19.78,58.0,X-LC,3.68,213898.0,9398.0,13903.0,3.19,4.60,6.50,11.39,10.0,0.68,1.51,13.96,X40N,ATH,PHARMA
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53764.0,-11242.0,162168.0,3.15,-17.29,301.63,232.18,198.0,-0.07,0.38,3.15,XY24,NTT,MISC
14,BLUESTARCO,2080.00,12.85,59.0,X-MC,3.22,194850.0,30180.0,13152.0,2.97,18.33,6.75,26.31,89.0,2.29,1.38,27.77,X40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,50.0,M-MC,12.66,227476.0,2514.0,165148.0,-3.20,1.12,72.60,74.53,192.0,0.02,1.61,33.73,XY24,BTT,STEEL
72,TATAELXSI,9161.00,-24.05,46.0,H-MC,6.65,102771.0,-29064.0,71292.0,-2.94,-22.05,69.37,32.03,98.0,-0.41,0.73,15.11,OX40N,NTT,IT
32,HINDZINC,730.22,33.63,66.0,M-LC,7.83,218066.0,12990.0,99591.0,-2.14,6.33,45.67,54.89,52.0,0.13,1.54,31.96,X5K,ATH,METALS
82,VALIANTORG,838.00,-290.89,30.0,H-SC,1.68,106267.0,-57338.0,178656.0,-2.07,-35.05,168.12,74.15,139.0,-0.32,0.75,36.22,XR,NTT,CHEMICALS
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.28,119165.0,18661.0,10451.0,-2.05,18.57,8.77,28.96,79.0,1.79,0.84,58.84,MH,ATH,METALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,108.87,64.0,H-SC,13.79,131598.0,-10881.0,30399.0,1.10,-7.64,23.10,13.70,163.0,-0.36,0.93,55.32,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.61,52.0,H-SC,1.95,222948.0,-46719.0,83048.0,0.97,-17.32,37.25,13.47,138.0,-0.56,1.58,13.36,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,39.0,M-SC,1.01,83185.0,-17592.0,17685.0,-0.32,-17.46,21.26,0.09,245.0,-0.99,0.59,9.29,OX40N,NTT,DURABLES
18,CERA,9475.0,-24.58,33.0,H-SC,1.81,137425.0,-38478.0,80504.0,-0.80,-21.87,58.58,23.89,149.0,-0.48,0.97,18.68,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.05,46.0,H-MC,6.65,102771.0,-29064.0,71292.0,-2.94,-22.05,69.37,32.03,98.0,-0.41,0.73,15.11,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,50.0,M-MC,12.66,227476.0,2514.0,165148.0,-3.20,1.12,72.60,74.53,192.0,0.02,1.61,33.73,XY24,BTT,STEEL
32,HINDZINC,730.22,33.63,66.0,M-LC,7.83,218066.0,12990.0,99591.0,-2.14,6.33,45.67,54.89,52.0,0.13,1.54,31.96,X5K,ATH,METALS
79,UNIONBANK,163.00,-11.63,57.0,M-LC,9.18,158430.0,17590.0,26901.0,1.10,12.49,16.98,31.59,66.0,0.65,1.12,42.29,XY24,NTT,BANKS
86,WIPRO,420.00,-11.98,56.0,M-LC,5.71,154436.0,3491.0,105959.0,1.09,2.31,68.61,72.51,53.0,0.03,1.09,8.47,XR,NTT,IT
11,BANKINDIA,190.00,-24.22,68.0,H-MC,11.95,195172.0,15364.0,97040.0,0.88,8.54,49.72,62.52,88.0,0.16,1.38,45.54,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.28,119165.0,18661.0,10451.0,-2.05,18.57,8.77,28.96,79.0,1.79,0.84,58.84,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,68.0,H-MC,11.95,195172.0,15364.0,97040.0,0.88,8.54,49.72,62.52,88.0,0.16,1.38,45.54,XR,NTT,BANKS
32,HINDZINC,730.22,33.63,66.0,M-LC,7.83,218066.0,12990.0,99591.0,-2.14,6.33,45.67,54.89,52.0,0.13,1.54,31.96,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.45,43.0,H-SC,8.74,127200.0,3864.0,127760.0,-0.51,3.13,100.44,106.72,119.0,0.03,0.90,27.24,AR,ATH,MISC
86,WIPRO,420.00,-11.98,56.0,M-LC,5.71,154436.0,3491.0,105959.0,1.09,2.31,68.61,72.51,53.0,0.03,1.09,8.47,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.29,35.0,X-LC,5.55,206342.0,-45426.0,98631.0,0.36,-18.04,47.80,21.13,27.0,-0.46,1.46,10.99,X40,ATH,PAINTS
23,DABUR,735.00,-12.30,35.0,X-MC,3.75,237359.0,-10709.0,119107.0,0.82,-4.32,50.18,43.70,102.0,-0.09,1.68,9.51,XY24,BTT,FMCG
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53764.0,-11242.0,162168.0,3.15,-17.29,301.63,232.18,198.0,-0.07,0.38,3.15,XY24,NTT,MISC
56,RELAXO,1176.00,-39.88,38.0,X-SC,2.54,81940.0,-63220.0,139151.0,-0.47,-43.55,169.82,52.31,136.0,-0.45,0.58,9.68,X40N,NTT,FOOTWEAR
22,COLPAL,3767.14,-4.76,39.0,X-MC,7.27,222400.0,-40965.0,154323.0,0.70,-15.55,69.39,43.04,84.0,-0.27,1.57,3.25,XY25,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-5.61,40.0,X-MC,0.56,233922.0,-13809.0,88867.0,0.92,-5.57,37.99,30.30,92.0,-0.16,1.65,6.98,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,1.72,63.0,X-MC,1.10,213405.0,21663.0,16091.0,2.95,11.30,7.54,19.69,99.0,1.35,1.51,18.86,XY25,NTT,AC
73,TATAMOTORS,1065.00,-22.16,43.0,X-LC,1.59,246441.0,-30082.0,140151.0,-0.32,-10.88,56.87,39.81,54.0,-0.21,1.74,18.11,XY24,NTT,AUTO
65,SIEMENS,4671.50,-1.97,46.0,H-LC,1.66,157605.0,-28490.0,75966.0,-0.83,-15.31,48.20,25.51,15.0,-0.38,1.11,16.14,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-19.88,43.0,X-MC,1.80,137758.0,1625.0,29494.0,-1.12,1.19,21.41,22.86,91.0,0.06,0.97,13.96,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-21.10,40.0,X-LC,10.34,284261.0,-31581.0,144234.0,-0.08,-10.00,50.74,35.67,5.0,-0.22,2.01,2.47,X40N,ATH,FMCG
42,ITC,452.00,-39.61,41.0,X-LC,2.27,195551.0,-4587.0,25480.0,0.04,-2.29,13.03,10.44,4.0,-0.18,1.38,3.14,X40,NTT,FMCG
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53764.0,-11242.0,162168.0,3.15,-17.29,301.63,232.18,198.0,-0.07,0.38,3.15,XY24,NTT,MISC
22,COLPAL,3767.14,-4.76,39.0,X-MC,7.27,222400.0,-40965.0,154323.0,0.70,-15.55,69.39,43.04,84.0,-0.27,1.57,3.25,XY25,ATH,FMCG
9,BAJAJHFL,181.50,-14.48,39.0,X-MC,4.96,180384.0,-20122.0,117286.0,-0.09,-10.04,65.02,48.45,90.0,-0.17,1.28,4.02,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53764.0,-11242.0,162168.0,3.15,-17.29,301.63,232.18,198.0,-0.07,0.38,3.15,XY24,NTT,MISC
56,RELAXO,1176.00,-39.88,38.0,X-SC,2.54,81940.0,-63220.0,139151.0,-0.47,-43.55,169.82,52.31,136.0,-0.45,0.58,9.68,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.29,42.0,X-SC,4.43,96778.0,-31892.0,77190.0,-0.54,-24.79,79.76,35.20,219.0,-0.41,0.68,11.08,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.72,51.0,X-SC,1.44,108214.0,6210.0,37713.0,0.88,6.09,34.85,43.06,122.0,0.16,0.77,24.25,X40N,ATH,MISC
33,HONAUT,58357.33,-22.39,48.0,X-SC,2.13,109305.0,-18633.0,65769.0,0.54,-14.56,60.17,36.84,143.0,-0.28,0.77,11.90,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-27.72,55.0,X-LC,12.14,287800.0,-58156.0,126402.0,1.14,-16.81,43.92,19.73,1.0,-0.46,2.04,6.00,X40,ATH,IT
40,INFY,2275.00,-17.09,58.0,X-LC,7.78,322588.0,9700.0,161971.0,0.34,3.10,50.21,54.87,3.0,0.06,2.28,9.96,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,X-LC,2.27,195551.0,-4587.0,25480.0,0.04,-2.29,13.03,10.44,4.0,-0.18,1.38,3.14,X40,NTT,FMCG
83,VBL,671.64,-21.10,40.0,X-LC,10.34,284261.0,-31581.0,144234.0,-0.08,-10.00,50.74,35.67,5.0,-0.22,2.01,2.47,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,47.0,H-LC,7.50,248880.0,-12739.0,131956.0,0.75,-4.87,53.02,45.57,7.0,-0.10,1.76,5.90,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,55.0,L-SC,29.25,83904.0,-29645.0,69699.0,2.34,-26.11,83.07,35.27,268.0,-0.43,0.59,105.36,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,46.0,H-SC,10.41,91500.0,-6480.0,28081.0,1.26,-6.61,30.69,22.05,152.0,-0.23,0.65,33.00,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.28,119165.0,18661.0,10451.0,-2.05,18.57,8.77,28.96,79.0,1.79,0.84,58.84,MH,ATH,METALS
78,UJJIVANSFB,60.00,108.87,64.0,H-SC,13.79,131598.0,-10881.0,30399.0,1.10,-7.64,23.10,13.70,163.0,-0.36,0.93,55.32,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,67.0,M-SC,15.45,147555.0,-1665.0,145843.0,1.79,-1.12,98.84,96.62,208.0,-0.01,1.04,59.69,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,68.86,68.0,H-SC,9.12,155458.0,-23613.0,89342.0,2.56,-13.19,57.47,36.71,135.0,-0.26,1.10,54.25,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,67.0,M-SC,15.45,147555.0,-1665.0,145843.0,1.79,-1.12,98.84,96.62,208.0,-0.01,1.04,59.69,XY25,NTT,FINANCE
32,HINDZINC,730.22,33.63,66.0,M-LC,7.83,218066.0,12990.0,99591.0,-2.14,6.33,45.67,54.89,52.0,0.13,1.54,31.96,X5K,ATH,METALS
26,GREENPANEL,537.00,238.50,62.0,M-SC,4.73,151231.0,-31847.0,110822.0,1.59,-17.40,73.28,43.14,230.0,-0.29,1.07,42.25,XY24,NTT,MISC
58,REPCOHOME,880.00,-57.53,66.0,H-SC,2.78,244882.0,-41623.0,298952.0,2.72,-14.53,122.08,89.82,134.0,-0.14,1.73,29.56,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.11
1,25,43.40
2,50,75.20


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.04
LC    32.25
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.95
X40      14.60
X40N     11.27
XR       11.00
XY25     10.42
AR        8.13
OX40N     7.62
X200      1.74
MH        1.63
X5K       1.54
SR        1.10
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.51
X-LC    22.89
X-MC    17.19
M-SC    12.56
H-LC     4.61
H-MC     4.60
X-SC     4.52
M-LC     3.75
M-MC     1.61
L-SC     1.45
L-LC     1.00
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.73,-5.65,40.46
IT,12.84,-15.82,78.55
FINANCE,10.53,-13.37,68.11
MISC,6.95,-17.15,82.07
BANKS,6.43,-9.81,67.57
PAINTS,5.49,-19.00,36.74
ELECTRICAL,5.29,-9.83,48.36
INSURANCE,3.76,-4.03,42.91
AC,3.58,4.16,11.89


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3199669.0,22
XR,1356135.0,14
AR,1181022.0,9
X40,774559.0,10
X40N,643545.0,9
OX40N,571847.0,10
XY25,462838.0,7
SR,265524.0,2
X5K,99591.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3397780.0,25
M-SC,1267894.0,15
X-MC,1104592.0,12
X-LC,1004126.0,13
X-SC,541323.0,6
H-MC,359007.0,4
H-LC,286815.0,3
L-SC,256432.0,3
M-LC,232451.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1190556.0      6
           AR         808057.0      5
           XR         795176.0      7
M-SC       XY24       679457.0      6
X-MC       XY24       588415.0      4
X-LC       X40        482044.0      5
X-SC       X40N       301965.0      4
H-SC       SR         265524.0      2
           OX40N      265518.0      4
X-MC       X40        215325.0      4
X-LC       X40N       211142.0      3
H-LC       AR         207922.0      2
X-LC       XY24       206800.0      2
H-MC       XY24       180224.0      1
X-MC       XY25       170414.0      2
L-SC       XR         167977.0      2
M-MC       XY24       165148.0      1
M-SC       AR         165043.0      2
X-SC       XY24       162168.0      1
M-SC       OX40N      146582.0      4
           XY25       145843.0      1
           XR         130969.0      2
X-MC       X40N       130438.0      2
M-LC       XR         105959.0      1
X-LC       XY25       104140.0      3
M-LC       X5K         99591.0      1
H-MC       XR          97040.0      1
L-SC       OX40N       88455.0      1
H-LC       X200        78893.0      1
X-SC       X40         77190.0      1
H-SC       MH          72949.0      1
H-MC       OX40N       71292.0      1
L-MC       XR          59014.0      1
L-LC       XY25        42441.0      1
M-LC       XY24        26901.0      1
H-MC       MH          10451.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
